In [5]:
import pandas as pd
import numpy as np
from neuralforecast import NeuralForecast
from neuralforecast.models import PatchTST
import logging
import os
from ticker_data import get_df_for_period
# Set both HTTP and HTTPS proxies

#ticker_data = {t: get_ticker(t, start=_start, end=_end) for t in tickers}
predict_ticker = 'AAPL'

tickers = ['AAPL', '^SPX', 'MSFT', '^NDX']
# NDX is an index: volume=0, and High/Low/Open == Close (no intraday range)
#tickers_with_vol  = ['AAPL', '^SPX', '^NDX', 'MSFT']
#tickers_with_ohlc = ['AAPL', '^SPX', '^NDX', 'MSFT']

period_train = {'start': '2019-06-01', 'end': '2025-01-01'}
period_test = {'start': '2025-01-01', 'end': '2025-09-01'}

df_train = get_df_for_period(tickers, period_train)
df_test = get_df_for_period(tickers, period_test)
df_full = pd.concat([df_train, df_test]).sort_values(['unique_id', 'ds'])

print(f"Training endet am: {df_train['ds'].max()}")
print(f"Blind-Test startet am: {df_test['ds'].min()}")
print(df_test)
df_traded_ticker = df_test[df_test['unique_id'] == f'{predict_ticker}_price'].copy()
print(df_traded_ticker)
df_traded_ticker_open = df_test[df_test['unique_id'] == f'{predict_ticker}_open'].copy()
df_traded_ticker_close = df_test[df_test['unique_id'] == f'{predict_ticker}_close'].copy()
# Echte AAPL-Preise für P&L-Berechnung
#aapl_open = ticker_data[predict_ticker]['Open']
#aapl_close = ticker_data[predict_ticker]['Close']

# 3. DIE SCHLEIFE
test_dates = df_test.index

[*********************100%***********************]  4 of 4 completed
/home/aidev/patchtst_timeseries/.venv/lib/python3.13/site-packages/pandas/core/internals/blocks.py:347: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
[*********************100%***********************]  4 of 4 completed


unique_id
SPX_vol    2
unique_id
AAPL_price    1405
AAPL_open     1405
AAPL_high     1405
AAPL_low      1405
AAPL_vol      1405
SPX_price     1405
SPX_open      1405
SPX_high      1405
SPX_low       1405
MSFT_price    1405
MSFT_open     1405
MSFT_high     1405
MSFT_low      1405
MSFT_vol      1405
NDX_price     1405
NDX_open      1405
NDX_high      1405
NDX_low       1405
NDX_vol       1405
SPX_vol       1403
Name: count, dtype: int64
Total days: 1405  |  Total rows: 28098  |  Series: 20
y range: [-1.772503, 1.562764]  |  mean: 0.000653  |  std: 0.108334
unique_id
AAPL_price    164
AAPL_open     164
AAPL_high     164
AAPL_low      164
AAPL_vol      164
SPX_price     164
SPX_open      164
SPX_high      164
SPX_low       164
SPX_vol       164
MSFT_price    164
MSFT_open     164
MSFT_high     164
MSFT_low      164
MSFT_vol      164
NDX_price     164
NDX_open      164
NDX_high      164
NDX_low       164
NDX_vol       164
Name: count, dtype: int64
Total days: 164  |  Total rows: 3280  |  Se

In [ ]:
%%capture

portfolio_value = 10000.0
current_pos = 0
position_return = 0
entry_price = None
THRESHOLD = 0.008
FEE = 0.0005
log_file = 'from_trained_test_ohlc.csv'
STOPLOSS_THRESHOLD = -0.01
SIGNAL_TRIGGER_STOPLOSS = -2
SIGNAL_TRIGGER_TP = 2
MODEL_INPUT_LEN = 80
# Log-Header
with open(log_file, 'w') as f:
    f.write("date,pred_momentum,signal,in_market,trade_return,portfolio_value,position_return\n")

#load model
nf = NeuralForecast.load(path='./checkpoints/patchtst_momentum_model_multivar_80days/')
logging.getLogger("pytorch_lightning").setLevel(logging.ERROR) 
logging.getLogger("neuralforecast").setLevel(logging.ERROR) 
logging.getLogger("lightning.pytorch").setLevel(logging.ERROR) 
trade = None
for i, today in enumerate(test_dates):
    # --- DATEN-FENSTER FÜR HEUTE ERSTELLEN ---
    df_list = []
    #we need a window MODEL_INPUT_LEN back from today to pass to the prediction model


    df_step = pd.concat(df_list).dropna()
    print(df_step)    
    # Prediction für den nächsten Tag
    forecast = nf.predict(df=df_step)
    
    #print(forecast)
    # Wir nehmen den ersten Tag des Horizonts (t+1)
    pred_y1 = forecast.query(f"unique_id == '{predict_ticker}_price'").iloc[0]['PatchTST']
    pred_y2 = forecast.query(f"unique_id == '{predict_ticker}_price'").iloc[1]['PatchTST']
    #print(forecast)
    # Signal Logik
    signal = 0
    pred_momentum = pred_y1 + pred_y2
    if pred_momentum > THRESHOLD: signal = 1
    elif pred_momentum < -THRESHOLD: signal = -1
    
    # --- ABRECHNUNG MIT ECHTEN PREISEN ---
    trade_return = 0.0

    if current_pos != 0:
        close_today = aapl_close.loc[today]
        position_return = current_pos * (close_today - entry_price) / entry_price    
        #STOPLOSS
        if position_return < STOPLOSS_THRESHOLD:
            signal  = SIGNAL_TRIGGER_STOPLOSS
    else:
        position_return = 0
    
    this_positon_return = position_return #needed for logging    
    day_current_signal = signal #needed for logging    
    day_current_pos = current_pos
    portfolio_value *= (1 + this_positon_return)
    #if signal != 0: # keep position open if there is no signal
    if True:    
        if signal != current_pos:
            # 1. Bestehende Position schließen zum heutigen Close
            if current_pos != 0 and entry_price is not None:
                #close_today = aapl_close.loc[today]
                #trade_return = current_pos * (close_today - entry_price) / entry_price
                #portfolio_value *= (1 + trade_return)
                position_return = 0
                signal = 0
                current_pos = 0
            # 2. Kosten bei Signalwechsel (Roundtrip-Gebühr)
            portfolio_value *= (1 - FEE)
            
            # 3. Neue Position eröffnen zum Close of today !
            if signal != 0 and i + 1 < len(test_dates):
                next_day = test_dates[i + 1]
                entry_price = aapl_open.loc[next_day]
            #    entry_price = aapl_close.loc[today]     
                current_pos = signal     
            else:
                entry_price = None
                        
    
    print(f"{today.date()} | Pred: {pred_momentum:.5f} | Signal: {signal} | "
          f"Pos: {current_pos} | Return: {trade_return:+.4f} | Port: {portfolio_value:.2f}")
    with open(log_file, 'a') as f:
        f.write(f"{today.date()},{pred_momentum},{day_current_signal},{day_current_pos},{trade_return},{portfolio_value},{this_positon_return}\n")

# Letzte offene Position zum letzten Close schließen
if current_pos != 0 and entry_price is not None:
    last_close = aapl_close.loc[test_dates[-1]]
    final_return = current_pos * (last_close - entry_price) / entry_price
    portfolio_value *= (1 + final_return)
    portfolio_value *= (1 - FEE)
    print(f"\nFinal close | Return: {final_return:+.4f} | Port: {portfolio_value:.2f}€")

print(f"\n=== Endwert: {portfolio_value:.2f} (Start: 10000€, Rendite: {(portfolio_value/10000-1)*100:+.2f}%) ===")

Seed set to 1


KeyError: 'AAPL'